In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Loading (remote) E-OBS observations and summer days (SU) calculation

Git repo: https://github.com/rmsryu/spark-climate-data.git
Data set: https://www.ecad.eu/download/ensembles/download.php

E-OBS data is loaded from the remote OpenDAP repository at KNMI. Please, note that when new versions become available the link is no longer valid since the URL is updated with the version number. Therefore, if the code below doesn't work please check the current E-OBS version and update the URL (https://www.ecad.eu/download/ensembles/download.php). 

# Study of daily precipitation from Station Aalsmeer Netherlands
Data source: https://climexp.knmi.nl/data/rrrr458.dat
Data provide by: Royal Netherlands Meteorological Institute (KNMI)

In [2]:
import numpy as np
import pandas as pd
from pyspark import SparkFiles
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
hadoopUrl = 'hdfs://hadoop-vm.internal.cloudapp.net:9000'
data_url = 'https://climexp.knmi.nl/data/rrrr458.dat'

spark.sparkContext.addFile(data_url)
df = spark.read.text(SparkFiles.get('rrrr458.dat'))
df = df.filter(df['value'].substr(0, 1) != '#') \
    .filter(df['value'].substr(0, 1) != '%') \
    .withColumn('date', to_date(substring(df['value'], 1, 8), 'yyyyMMdd')) \
    .withColumn('precipitation', split(df['value'], '\s+')[1]) \
    .select('date', 'precipitation') \
    .orderBy('date')



In [4]:
# save file to hdfs
df.write.mode("overwrite").parquet(f"{hadoopUrl}/precipitation/rrrr458")